In [2]:
import nglview as nv
import numpy as np
from nglview import NGLWidget
view = nv.show_file("C:/mdpath/first_frame.pdb")
view.display(gui= True, style= "ngl")



NGLWidget(gui_style='ngl')

In [ ]:
import json

def precompute_path_properties(json_data, colors):
    cluster_colors = {}
    color_index = 0
    path_properties = []

    for clusterid, cluster in json_data.items():
        cluster_colors[clusterid] = colors[color_index % len(colors)]
        color_index += 1
        coord_pair_counts = {}
        path_number = 1

        for pathway_index, pathway in enumerate(cluster):
            for i in range(len(pathway) - 1):
                coord1 = pathway[i][0]
                coord2 = pathway[i + 1][0]
                if isinstance(coord1, list) and isinstance(coord2, list) and len(coord1) == 3 and len(coord2) == 3:
                    coord_pair = (tuple(coord1), tuple(coord2))
                    if coord_pair not in coord_pair_counts:
                        coord_pair_counts[coord_pair] = 0
                    coord_pair_counts[coord_pair] += 1
                    radius = 0.015 + 0.015 * (coord_pair_counts[coord_pair] - 1)
                    color = cluster_colors[clusterid]

                    path_properties.append({
                        "clusterid": clusterid,
                        "pathway_index": pathway_index,
                        "path_segment_index": i,
                        "coord1": coord1,
                        "coord2": coord2,
                        "color": color,
                        "radius": radius,
                        "path_number": path_number
                    })

                    path_number += 1
                else:
                    print(f"Ignoring pathway {pathway} as it does not fulfill the coordinate format.")
    return path_properties

def generate_ngl_script(path_properties):
    for prop in path_properties:
        shape_script = f"""
            var shape = new NGL.Shape('custom_shape_{prop["clusterid"]}_{prop["pathway_index"]}_{prop["path_segment_index"]}');
            shape.addCylinder([{prop["coord1"][0]}, {prop["coord1"][1]}, {prop["coord1"][2]}], 
                              [{prop["coord2"][0]}, {prop["coord2"][1]}, {prop["coord2"][2]}], 
                              [{prop["color"][0]}, {prop["color"][1]}, {prop["color"][2]}], 
                              {prop["radius"]});
            var shapeComp = this.stage.addComponentFromObject(shape);
            shapeComp.addRepresentation('buffer');
        """
        if view:
            view._execute_js_code(shape_script)
        else:
            print("View is not defined.")

# Load JSON data
with open("C:/mdpath/clusters_paths.json", 'r') as json_file:
    json_data = json.load(json_file)

# Define colors
colors = [
    [1, 0, 0],  # Red
    [0, 1, 0],  # Green
    [0, 0, 1],  # Blue
    [1, 1, 0],  # Yellow
    [1, 0, 1],  # Magenta
    [0, 1, 1],  # Cyan
    [0.5, 0.5, 0.5],  # Gray
    [1, 0.5, 0],  # Orange
    [0.5, 0, 0.5],  # Purple
    [0.5, 1, 0.5],  # Light Green
]

# Precompute path properties
path_properties = precompute_path_properties(json_data, colors)

# Generate NGL script using precomputed properties
generate_ngl_script(path_properties)


In [3]:
import json

# Load precomputed JSON data
with open("C:/mdpath/precomputed_clusters_paths.json", 'r') as json_file:
    precomputed_data = json.load(json_file)

def generate_ngl_script(precomputed_data):
    for prop in precomputed_data:
        shape_script = f"""
            var shape = new NGL.Shape('custom_shape_{prop["clusterid"]}_{prop["pathway_index"]}_{prop["path_segment_index"]}');
            shape.addCylinder([{prop["coord1"][0]}, {prop["coord1"][1]}, {prop["coord1"][2]}], 
                              [{prop["coord2"][0]}, {prop["coord2"][1]}, {prop["coord2"][2]}], 
                              [{prop["color"][0]}, {prop["color"][1]}, {prop["color"][2]}], 
                              {prop["radius"]});
            var shapeComp = this.stage.addComponentFromObject(shape);
            shapeComp.addRepresentation('buffer');
        """
        if view:
            view._execute_js_code(shape_script)
        else:
            print("View is not defined.")

# Generate NGL script using precomputed properties
generate_ngl_script(precomputed_data)
